<a href="https://colab.research.google.com/github/lucienrivat/PommePOire/blob/main/PommePoir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Import dataset
import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import datetime
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow as tf
import pathlib
import os
import zipfile
import random

# Télécharger le fichier ZIP Apple depuis GitHub
apple_zip = tf.keras.utils.get_file(
    "Apple.zip",
    "https://github.com/lucienrivat/PommePOire/raw/main/Apple.zip",
    extract=False
)

# Télécharger le fichier ZIP Pear depuis GitHub
pear_zip = tf.keras.utils.get_file(
    "Pear.zip",
    "https://github.com/lucienrivat/PommePOire/raw/main/pear.zip",
    extract=False
)

# Extraire les fichiers ZIP dans /content/datasets
with zipfile.ZipFile(apple_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets/')

with zipfile.ZipFile(pear_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets/')

# Définir les chemins des dossiers contenant les images
apple_data_dir = pathlib.Path('/content/datasets/')
pear_data_dir = pathlib.Path('/content/datasets/pear/')

print("Dossier des images Apple :", apple_data_dir)
print("Dossier des images Pear :", pear_data_dir)
print("Chemin absolu Apple :", os.path.abspath(apple_data_dir))
print("Chemin absolu Pear :", os.path.abspath(pear_data_dir))

# Compter le nombre d'images dans chaque dataset
apple_image_count = len(list(apple_data_dir.glob('*/*')))
pear_image_count = sum([len(list(pear_data_dir.glob(f'{folder}/*'))) for folder in os.listdir(pear_data_dir)])

print("Nombre d'images Apple trouvées :", apple_image_count)
print("Nombre d'images Pear trouvées :", pear_image_count)

# Définir les paramètres du dataset
batch_size = 3
img_height = 200
img_width = 200

# Charger les datasets Apple et Pear
train_apple_data = tf.keras.preprocessing.image_dataset_from_directory(
  apple_data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
)

val_apple_data = tf.keras.preprocessing.image_dataset_from_directory(
  apple_data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
)

# Pour Pear, on doit choisir un sous-dossier
pear_subfolders = [folder for folder in os.listdir(pear_data_dir) if os.path.isdir(os.path.join(pear_data_dir, folder))]
print("Sous-dossiers dans Pear:", pear_subfolders)

# Sélectionner six images aléatoires
images_to_display = []

for _ in range(6):
    # Choisir si l'image provient d'Apple ou de Pear
    source = random.choice(['apple', 'pear'])

    if source == 'apple':
        # Choisir une image aléatoire dans Apple
        random_apple_image = random.choice(list(apple_data_dir.glob('*.*')))
        img_apple = mpimg.imread(random_apple_image)
        images_to_display.append(img_apple)
    else:
        # Choisir un sous-dossier de manière aléatoire dans Pear
        chosen_pear_folder = random.choice(pear_subfolders)
        pear_images = list(pathlib.Path(pear_data_dir, chosen_pear_folder).glob('*.*'))

        # Sélectionner une image aléatoire du sous-dossier Pear
        random_pear_image = random.choice(pear_images)
        img_pear = mpimg.imread(random_pear_image)
        images_to_display.append(img_pear)

# Afficher les images
plt.figure(figsize=(15, 10))
for i, img in enumerate(images_to_display):
    ax = plt.subplot(2, 3, i + 1)
    plt.imshow(img)
    plt.axis("off")
    if i < 3:
        plt.title(f"Apple {i + 1}")
    else:
        plt.title(f"Pear {i - 2}")
plt.show()


Dossier des images Apple : /content/datasets
Dossier des images Pear : /content/datasets/pear
Chemin absolu Apple : /content/datasets
Chemin absolu Pear : /content/datasets/pear
Nombre d'images Apple trouvées : 466
Nombre d'images Pear trouvées : 5252
Found 5708 files belonging to 2 classes.
Using 4567 files for training.
Found 5708 files belonging to 2 classes.
Using 1141 files for validation.
Sous-dossiers dans Pear: ['Pear Williams 1', 'Pear Abate 1', 'Pear Red 1', 'Pear 3', 'Pear Stone 1', 'Pear 1', 'Pear 2', 'Pear Forelle 1', 'Pear Monster 1', 'Pear Kaiser 1']


IndexError: Cannot choose from an empty sequence